VariantSpark integration with Hail 0.2
==============================

## Bootstrap

Use `vshl.init()` to include `variant-spark` jar on the classpath. 

In [1]:
import hail as hl
import varspark.hail as vshl
vshl.init()

using variant-spark jar at '/home/brendan/code/VariantSpark/target/variant-spark_2.11-0.3.0-SNAPSHOT-all.jar'


TypeError: 'JavaPackage' object is not callable

In [ ]:
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

# Load and explore hipster data

In [ ]:
data = hl.import_vcf('../data/hipsterIndex/hipster.vcf.bgz')

In [ ]:
labels = hl.import_table('../data/hipsterIndex/hipster_labels.txt', delimiter=',', 
                types=dict(label='float64', score='float64')).key_by('samples')

In [ ]:
mt = data.annotate_cols(hipster = labels[data.s])
mt.describe()

In [ ]:
mt.count()

## Run log regression using Hail

In [ ]:
gwas = hl.logistic_regression_rows(test='score',
                                y=mt.hipster.label,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0],
                                 pass_through=[mt.rsid])

In [ ]:
gwas.show(3)

In [ ]:
p = hl.plot.manhattan(gwas.p_value, hover_fields=dict(rs=gwas.rsid))
show(p)

_Fig 1: Manhattan plot for logistic regression p-values._

## Build random forest and extract gini importanct with VariantSpark (on the same data)

In [ ]:
rf_model = vshl.random_forest_model(y=mt.hipster.label,
                    x=mt.GT.n_alt_alleles())
rf_model.fit_trees(500, 100)

In [ ]:
print(rf_model.oob_error())
impTable = rf_model.variable_importance()
impTable.show(3)

Join hail and VariantSpark results (this is only needed here to get the RSID's)

In [ ]:
gwas_with_imp = gwas.join(impTable)

In [ ]:
import varspark.hail.plot as vshlplt
p = vshlplt.manhattan_imp(gwas_with_imp.importance, 
                            hover_fields=dict(ri=gwas_with_imp.rsid),
                            significance_line = None)
show(p)

_Fig 2: Manhattan plot for rf gin importance values._

## Compare logistc regression values vs. rf importance

In [ ]:
p = hl.plot.scatter(x=-hl.log10(gwas_with_imp.p_value),
                    y=gwas_with_imp.importance, 
                    xlabel = '-log10(p-value)',
                    ylabel = 'gini importance',
                    hover_fields=dict(rs=gwas_with_imp.rsid, loc=gwas_with_imp.locus))
show(p)

_Fig 3: Compare gini importance vs logistic regresion p-values._